In [ ]:
!pip install boto3

In [8]:
from pathlib import Path

import boto3
import pandas as pd
from botocore.config import Config
from botocore.exceptions import NoCredentialsError

In [3]:
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
import os

In [9]:
ENDPOINT = os.getenv("PIKNIK_ENDPOINT")
PROVIDER = os.getenv("PIKNIK_PROVIDER")

In [6]:
ACCESS_KEY = os.getenv("PIKNIK_ACCESS_KEY_ID")
SECRET_KEY = os.getenv("PIKNIK_SECRET_ACCESS_KEY")

In [11]:
s3 = boto3.client(
    "s3",
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY,
    endpoint_url="http://" + ENDPOINT,
)

In [19]:
s3 = boto3.resource(
    "s3",
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY,
    endpoint_url="http://" + ENDPOINT,
)

In [20]:
s3.list_buckets()

AttributeError: 's3.ServiceResource' object has no attribute 'list_buckets'

In [21]:
my_bucket = s3.Bucket("uofm-data1")

In [23]:
for obj in my_bucket.objects.filter(Prefix="landsat9-c2-l1/20211031"):
    print(obj.key)

landsat9-c2-l1/20211031/LC09_L1GT_019032_20211031_20220118_02_T2/LC09_L1GT_019032_20211031_20220118_02_T2_ANG.txt
landsat9-c2-l1/20211031/LC09_L1GT_019032_20211031_20220118_02_T2/LC09_L1GT_019032_20211031_20220118_02_T2_B1.TIF
landsat9-c2-l1/20211031/LC09_L1GT_019032_20211031_20220118_02_T2/LC09_L1GT_019032_20211031_20220118_02_T2_B10.TIF
landsat9-c2-l1/20211031/LC09_L1GT_019032_20211031_20220118_02_T2/LC09_L1GT_019032_20211031_20220118_02_T2_B11.TIF
landsat9-c2-l1/20211031/LC09_L1GT_019032_20211031_20220118_02_T2/LC09_L1GT_019032_20211031_20220118_02_T2_B2.TIF
landsat9-c2-l1/20211031/LC09_L1GT_019032_20211031_20220118_02_T2/LC09_L1GT_019032_20211031_20220118_02_T2_B3.TIF
landsat9-c2-l1/20211031/LC09_L1GT_019032_20211031_20220118_02_T2/LC09_L1GT_019032_20211031_20220118_02_T2_B4.TIF
landsat9-c2-l1/20211031/LC09_L1GT_019032_20211031_20220118_02_T2/LC09_L1GT_019032_20211031_20220118_02_T2_B5.TIF
landsat9-c2-l1/20211031/LC09_L1GT_019032_20211031_20220118_02_T2/LC09_L1GT_019032_20211031_20

In [ ]:
my_config = Config(
    region_name="us-west-1",
    signature_version="v4",
    retries={"max_attempts": 10, "mode": "standard"},
)

In [ ]:
ACCESS_KEY = ""
SECRET_KEY = ""

PATH_TO_CSV = ""
car_dir = Path()

s3 = boto3.client(
    "s3",
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY,
    endpoint_url=ENDPOINT,
)

df = pd.read_csv(PATH_TO_CSV)


def upload_to_aws(local_file, bucket, s3_file):

    try:
        s3.upload_file(local_file, bucket, s3_file)
        print("Upload Successful")
        return True
    except FileNotFoundError:
        print("The file was not found")
        return False
    except NoCredentialsError:
        print("Credentials not available")
        return False


for idx, row in df.iterrows():
    print("Index ", idx)
    print(row["name"])
    car_target = car_dir / str(row["file"])

    if not car_target.exists():
        print(row["name"])
        print("does not exists")
        continue

    print(str(car_target), "<NAME>", row["file"])
    uploaded = upload_to_aws(str(car_target), "<NAME>", row["file"])

    if not uploaded:
        print(row["name"])
        print("not uploaded")